# [Problem](https://thefiddler.substack.com/p/can-you-shut-the-box)

_In a simplified version of Shut the Box, there are six initially unflipped tiles, numbered 1-6, on the side of a box. You roll a fair, six-sided die, after which you can flip down any combination of unflipped tiles that add to the roll. For example, if you roll a 5, you could flip down the 5, the 1 and the 4, or the 2 and the 3. You proceed until no set of flips is possible (e.g., you rolled a 3 but your only remaining tiles are 2 and 6). To win, you must flip down all six tiles._

_Assuming you play with an optimal strategy (i.e., maximizing your chances of winning after any given roll), what is the probability that you’ll win?_

# Solution [Incorrect]

First things first, if we want to find the probability, we'll first need to first determine optimal play. Intuitively, we should always try to flip the tile of the number we rolled (i.e. largest possible tiles we can flip) since the larger numbers are harder to flip (6 can only be flipped if you roll a 6 while 1 can be flipped on any roll except for a 2). But what if the number you rolled isn't available? While I'm not sure of this, I think the optimality of the choice is only dependent on the number of tiles you file, the goal being to flip as few as possible and to flip the largest numbers.

In order to win, we know we must roll a total of $21$. Let's go backwards starting with all tiles flipped to figure out which rolls work

$ S = \{1,2,3,4,5,6\} $

To get the willing roll combinations, we can combine elements such that we remove the two elements we combine, add their sum to the set, and only perform this operation if the sum is $\leq 6$. For example

$ S = \{2,4,4,5,6\} \text{ combination of 1 and 3}$

$ S = \{3,6,6,6\} \text{ combination of (1,5) and (2,4)}$

are valid.

After playing around for a while trying to figure out some nice analytic way of doing this, I finally gave up and used a computer to generate all of these combinations. We will want to make sure though that we are efficient with our implementation since naive implementations can easily result in very high time complexities. 

In [10]:
from itertools import combinations
from collections import Counter

def generate_all_sets_custom_range(n_l, n_h, k):
    """
    Generates all possible unique sets by combining elements of S = {n_l, n_l+1, ..., n_h}
    according to the specified combination rules with threshold k.

    Parameters:
    n_l (int): The lower bound integer in the initial set S.
    n_h (int): The upper bound integer in the initial set S. Must satisfy n_h >= n_l.
    k (int): The maximum allowed sum for any two elements to be combined.

    Returns:
    set of tuples: All unique sets generated during the combination process.
    """
    # Validate inputs
    if not isinstance(n_l, int) or not isinstance(n_h, int) or not isinstance(k, int):
        raise TypeError("All inputs must be integers.")
    if n_h < n_l:
        raise ValueError("Upper bound n_h must be greater than or equal to lower bound n_l.")
    if k < (2 * n_l):
        # If the smallest possible sum exceeds k, no combinations are possible
        print("No combinations possible since the smallest possible sum exceeds k.")
    
    # Initialize the starting set S = {n_l, n_l+1, ..., n_h}
    initial_set = list(range(n_l, n_h + 1))
    all_sets = set()          # To store all unique sets
    processed = set()         # To keep track of already processed sets

    def recurse(current_counter):
        # Convert the current Counter to a sorted tuple for hashing
        sorted_set = tuple(sorted(current_counter.elements()))
        
        if sorted_set in processed:
            return
        processed.add(sorted_set)
        all_sets.add(sorted_set)
        
        # Flag to check if any combination is possible
        combination_possible = False
        
        # Generate all unique pairs to attempt combination
        unique_elements = list(current_counter.elements())
        for a, b in combinations(unique_elements, 2):
            s = a + b
            if s <= k:
                combination_possible = True
                # Create a new Counter: remove a and b, add their sum
                new_counter = current_counter.copy()
                new_counter[a] -= 1
                new_counter[b] -= 1
                if new_counter[a] == 0:
                    del new_counter[a]
                if new_counter[b] == 0:
                    del new_counter[b]
                new_counter[s] += 1
                
                # Recursively process the new set
                recurse(new_counter)
        
        # If no further combinations are possible, continue
        # (Since we're collecting all sets, no action needed here)

    # Start the recursion with the initial set represented as a Counter
    initial_counter = Counter(initial_set)
    recurse(initial_counter)
    
    return all_sets

n_l = 1  # Lower bound (can be any integer)
n_h = 6  # Upper bound (must be >= n_l)
k = 6    # Combination threshold

# Generate all sets
result = generate_all_sets_custom_range(n_l, n_h, k)

# Sort the results for better readability
sorted_result = sorted(result, key=lambda x: (len(x), x))

print(f"All possible resulting sets for S = {{{n_l}, {', '.join(map(str, range(n_l +1, n_h +1)))} }} with combination threshold k = {k}:")
print(f"Total number of unique sets: {len(sorted_result)}")
for res_set in sorted_result:
    print(res_set)


All possible resulting sets for S = {1, 2, 3, 4, 5, 6 } with combination threshold k = 6:
Total number of unique sets: 10
(3, 6, 6, 6)
(4, 5, 6, 6)
(5, 5, 5, 6)
(1, 3, 5, 6, 6)
(1, 4, 5, 5, 6)
(2, 3, 4, 6, 6)
(2, 3, 5, 5, 6)
(2, 4, 4, 5, 6)
(3, 3, 4, 5, 6)
(1, 2, 3, 4, 5, 6)


In [11]:
# Function to compute the multinomial coefficient
def multinomial_coefficient(s):
    """
    Computes the multinomial coefficient for the given set s.

    Parameters:
    s (tuple): The set for which to compute the multinomial coefficient.

    Returns:
    int: The computed multinomial coefficient.
    """
    # Compute the factorial of the sum of all elements
    numerator = 1
    for i in s:
        numerator *= i
    denominator = 1
    for i in Counter(s).values():
        denominator *= i
    return numerator // denominator

In [1]:
# Compute the multinomial coefficient for each set
coefficients = [multinomial_coefficient(s) for s in sorted_result]

# Compute the probability
# prob = sum(coefficients * (1/6)**len(s) for s, coefficients in zip(sorted_result, coefficients))
prob = sum(coefficients) / (n_h - n_l + 1)**(n_h - n_l + 1)
print(f"\nProbability of generating a set with sum <= {k}: {prob:.6f}")

NameError: name 'sorted_result' is not defined